In [2]:
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import LinearLocator, FormatStrFormatter

def plotSol(u, n):
    u = np.reshape(u, (n, n))
    
    X = np.arange(0, 1, 1 / n)
    Y = np.arange(0, 1, 1 / n)
    X, Y = np.meshgrid(X, Y)

    fig = plt.figure()
    ax = fig.gca(projection='3d')
    
    surf = ax.plot_surface(X, Y, u, cmap=cm.hot)
    
    plt.show()

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
xc = [0.5, 0.5]

def g(x):
    if ((x[0]-xc[0])**2+(x[1]-xc[1])**2 < np.log(2)/10):
        return 0
    return 1 - 4*np.exp(-20*np.linalg.norm(np.subtract(x, xc))**2)

def plotg(dimension):
    temp = [[0 for x in range(dimension)] for y in range(dimension)] 
    for i in range(dimension):
        for j in range(dimension):
            temp[i][j] = g((i/dimension, j/dimension))
    plotSol(temp, dimension)
    
#i = y, j = x
def iStoC(i, j, dim):
        xcord = (j % dim) / dim
        ycord = (i // dim) / dim
        return (xcord, ycord)

#S[y][x]
def init_S(n, kappa):
    S = np.zeros((n*n, n*n))
    h = 1/n
    for i in range(n*n):
        gn = g(iStoC(i+1, i, n))
        ge = g(iStoC(i, i+1, n))
        gs = g(iStoC(i-1, i, n))
        gw = g(iStoC(i, i-1, n))
        if (i < n):
            if (i == 0):
                S[i][i] = -(ge+gn)/2-h*kappa
                S[i][i+1] = ge/2
                S[i][i+n] = gn/2
            elif(i == n - 1):
                S[i][i] = -(gw+gn)/2-h*kappa
                S[i][i-1] = gw/2
                S[i][i+n] = gn/2
            else:
                S[i][i] = -gn-(ge+gw)/2-h*kappa
                S[i][i-1] = gw/2
                S[i][i+1] = ge/2
                S[i][i+n] = gn
                
        if (i >= n*(n-1)):
            if (i % n == 0):
                S[i][i] = -(ge+gs)/2-h*kappa
                S[i][i+1] = ge/2
                S[i][i-n] = gs/2
            elif ((i + 1) % n == 0):
                S[i][i] = -(gw+gs)/2-h*kappa
                S[i][i-1] = gw/2
                S[i][i-n] = gs/2
            else:
                S[i][i] = -gs-(ge+gw)/2-h*kappa
                S[i][i-1] = gw/2
                S[i][i+1] = ge/2
                S[i][i-n] = gs
                
        if (n <= i < n*(n-1)):
            S[i][i-n] = gs
            S[i][i+n] = gn
            if (i % n == 0):
                S[i][i] = -ge-(gs+gn)/2-h*kappa
                S[i][i+1] = ge
            elif ((i + 1) % n == 0):
                S[i][i] = -gw -(gs+gn)/2-h*kappa
                S[i][i-1] = gw
            else:
                S[i][i] = -gn-ge-gs-gw
                S[i][i-1] = gw
                S[i][i+1] = ge
    return S

def init_M(dim):
    D = np.ones(dim)
    D[[0,-1]] = 0.5
    D = np.concatenate((0.5*D,np.tile(D,dim-2),0.5*D))
    M = np.diag(D)
    return (1/(dim*dim))*M

def init_u0(n):
    u = np.zeros((n*n, 1))
    for i in range(0, n*n):
        ycord = (i // n) / n
        xcord = (i % n) / n
        x = (xcord, ycord)
        if (np.linalg.norm(x) < 0.05):
            u[i] = 0.1
    return u

def backwardEuler(M, S, lam, dt, u0, epsilon, dimension, maxiter):
    Minv = np.linalg.inv(M)
    MinvS = Minv @ S
    ukn = ukn1 = u0
    sol = np.zeros((maxiter, dimension*dimension))
    for n in range(maxiter):
        print("iter", n)
        norm = np.inf
        while(norm > epsilon):
            uk1n1 = np.linalg.solve((np.eye(dimension*dimension) - dt*MinvS), (ukn + dt*lam*(ukn1 * (np.ones(ukn1.shape)-ukn1))))
            norm = np.linalg.norm(ukn1 - uk1n1)
            ukn1 = uk1n1
            #print(norm)
        ukn = ukn1
        
        sol[n,:] = ukn[:,0]
    
    for i in range(maxiter):
        if (i % 100 == 0):
            plotSol(sol[i], dimension)

def forwardEuler(M, S, lam, dt, u0, dimension, maxiter):
    Minv = np.linalg.inv(M)
    MinvS = Minv @ S
    
    un = un1 = u0
    sol = np.zeros((maxiter, dimension*dimension))
    for n in range(maxiter):
        un1 = un + dt*MinvS.dot(un) + dt*lam*(un*(1-un))
        un = un1
        sol[n,:] = un[:,0]
    
    for i in range(maxiter):
        if (i % 100 == 0):
            plotSol(sol[i], dimension)
    
#print(init_S(10, 1))
# plt.spy(init_S(6,1))
#plt.matshow(init_S(100,1))
dimension = 20
backwardEuler(init_M(dimension), init_S(dimension, 1), 10, 0.0001, init_u0(dimension), 1e-6, dimension, 2000)
#forwardEuler(init_M(dimension), init_S(dimension, 1), 10, 0.0000001, init_u0(dimension), dimension, 300000)
#plotg(50)
#plt.matshow(init_S(10, 1))

iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9
iter 10
iter 11
iter 12
iter 13
iter 14
iter 15
iter 16
iter 17
iter 18
iter 19
iter 20
iter 21
iter 22
iter 23
iter 24
iter 25
iter 26
iter 27
iter 28
iter 29
iter 30
iter 31
iter 32
iter 33
iter 34
iter 35
iter 36
iter 37
iter 38
iter 39
iter 40
iter 41
iter 42
iter 43
iter 44
iter 45
iter 46
iter 47
iter 48
iter 49
iter 50
iter 51
iter 52
iter 53
iter 54
iter 55
iter 56
iter 57
iter 58
iter 59
iter 60
iter 61
iter 62
iter 63
iter 64
iter 65
iter 66
iter 67
iter 68
iter 69
iter 70
iter 71
iter 72
iter 73
iter 74
iter 75
iter 76
iter 77
iter 78
iter 79
iter 80
iter 81
iter 82
iter 83
iter 84
iter 85
iter 86
iter 87
iter 88
iter 89
iter 90
iter 91
iter 92
iter 93
iter 94
iter 95
iter 96
iter 97
iter 98
iter 99
iter 100
iter 101
iter 102
iter 103
iter 104
iter 105
iter 106
iter 107
iter 108
iter 109
iter 110
iter 111
iter 112
iter 113
iter 114
iter 115
iter 116
iter 117
iter 118
iter 119
iter 120
iter 121
iter 122
ite

iter 930
iter 931
iter 932
iter 933
iter 934
iter 935
iter 936
iter 937
iter 938
iter 939
iter 940
iter 941
iter 942
iter 943
iter 944
iter 945
iter 946
iter 947
iter 948
iter 949
iter 950
iter 951
iter 952
iter 953
iter 954
iter 955
iter 956
iter 957
iter 958
iter 959
iter 960
iter 961
iter 962
iter 963
iter 964
iter 965
iter 966
iter 967
iter 968
iter 969
iter 970
iter 971
iter 972
iter 973
iter 974
iter 975
iter 976
iter 977
iter 978
iter 979
iter 980
iter 981
iter 982
iter 983
iter 984
iter 985
iter 986
iter 987
iter 988
iter 989
iter 990
iter 991
iter 992
iter 993
iter 994
iter 995
iter 996
iter 997
iter 998
iter 999
iter 1000
iter 1001
iter 1002
iter 1003
iter 1004
iter 1005
iter 1006
iter 1007
iter 1008
iter 1009
iter 1010
iter 1011
iter 1012
iter 1013
iter 1014
iter 1015
iter 1016
iter 1017
iter 1018
iter 1019
iter 1020
iter 1021
iter 1022
iter 1023
iter 1024
iter 1025
iter 1026
iter 1027
iter 1028
iter 1029
iter 1030
iter 1031
iter 1032
iter 1033
iter 1034
iter 1035
iter 1036
